# Hugging Face Transformers 微调语言模型-问答任务

我们已经学会使用 Pipeline 加载支持问答任务的预训练模型，本教程代码将展示如何微调训练一个支持问答任务的模型。

**注意：微调后的模型仍然是通过提取上下文的子串来回答问题的，而不是生成新的文本。**

#### 模型执行问答效果示例

![Widget inference representing the QA task](docs/images/question_answering.png)

In [2]:
# 根据你使用的模型和GPU资源情况，调整以下关键参数

# 根据自身下载不同的数据集
squad_v2 = False
# 模型名
model_checkpoint = "distilbert-base-uncased"
# 批次大小
batch_size = 16

# 下载数据集

在本教程中，我们将使用[斯坦福问答数据集(SQuAD）](https://rajpurkar.github.io/SQuAD-explorer/)。

#### SQuAD 数据集

**斯坦福问答数据集(SQuAD)** 是一个阅读理解数据集，由众包工作者在一系列维基百科文章上提出问题组成。每个问题的答案都是相应阅读段落中的文本片段或范围，或者该问题可能无法回答。

SQuAD2.0将SQuAD1.1中的10万个问题与由众包工作者对抗性地撰写的5万多个无法回答的问题相结合，使其看起来与可回答的问题类似。要在SQuAD2.0上表现良好，系统不仅必须在可能时回答问题，还必须确定段落中没有支持任何答案，并放弃回答。


### 下载数据集

In [3]:
# 加载数据集下载包

from datasets import load_dataset

In [4]:
# 根据配置判断 下载对应的数据集

dataset = load_dataset("squad_v2" if squad_v2 else "squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
# 查看数据集格式
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

#### 对比数据集

相比快速入门使用的 Yelp 评论数据集，我们可以看到 SQuAD 训练和测试集都新增了用于上下文、问题以及问题答案的列：

**YelpReviewFull Dataset：**

```json

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [7]:
# 查看数据集具体格式
dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

#### 从上下文中组织回复内容

我们可以看到答案是通过它们在文本中的起始位置（这里是515，注意：这里的515是以字符为单位），以及它们的完整文本表示的，这是上面提到的上下文的子字符串。  
例如 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

context: 文本数据  
question：问题  
answers：问题的答案，以及在文本中出现的位置

In [9]:
# 通过答案首个地址+答案长度从原始文本中查找答案
answer_start = dataset['train'][0]['answers']['answer_start'][0]
answer_end = answer_start + len(dataset['train'][0]['answers']['text'][0])
dataset['train'][0]['context'][answer_start:answer_end]

'Saint Bernadette Soubirous'

In [10]:
import numpy as np 
# 导入dataset的数据类型
from datasets import ClassLabel, Sequence, Value
import pandas as pd
from IPython.display import display, HTML

In [12]:
# 写一个功能 用来随机抽取数据集中的数据，并且以html的方式展示出来
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= dataset.num_rows, "Can't pick more elements than there are in the dataset."
    picks = np.random.choice(dataset.num_rows, size=num_examples, replace=True).tolist()
    # 转换成pd格式
    df = pd.DataFrame(dataset[picks])

    # for column, typ in dataset.features.items():
    #     if isinstance(typ, ClassLabel):
    #         df[column] = df[column].transform(lambda i: typ.names[i])
    #         print('------')
    #     elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
    #         print("+++")
    #         df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])

    display(HTML(df.to_html()))

In [13]:
# transform 可以将一个函数应用于 DataFrame 或 Series 的每个元素上。与 apply 方法不同，
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [5, 6, 7, 8]
})
df['A'] = df['A'].transform(lambda x:x+10)
df

,A,B
0,11,5
1,12,6
2,13,7
3,14,8


In [15]:
show_random_elements(dataset['train'],5)

,id,title,context,question,answers
0,570d9f0316d0071400510bfd,Adolescence,"A third gain in cognitive ability involves thinking about thinking itself, a process referred to as metacognition. It often involves monitoring one's own cognitive activity during the thinking process. Adolescents' improvements in knowledge of their own thinking patterns lead to better self-control and more effective studying. It is also relevant in social cognition, resulting in increased introspection, self-consciousness, and intellectualization (in the sense of thought about one's own thoughts, rather than the Freudian definition as a defense mechanism). Adolescents are much better able than children to understand that people do not have complete control over their mental activity. Being able to introspect may lead to two forms of adolescent egocentrism, which results in two distinct problems in thinking: the imaginary audience and the personal fable. These likely peak at age fifteen, along with self-consciousness in general.",What is the term used to describe thinking about thinking itself?,"{'text': ['metacognition'], 'answer_start': [100]}"
1,5726f9b0dd62a815002e96a3,Avicenna,"In 1980, the Soviet Union, which then ruled his birthplace Bukhara, celebrated the thousandth anniversary of Avicenna's birth by circulating various commemorative stamps with artistic illustrations, and by erecting a bust of Avicenna based on anthropological research by Soviet scholars.[citation needed] Near his birthplace in Qishlak Afshona, some 25 km (16 mi) north of Bukhara, a training college for medical staff has been named for him.[year needed] On the grounds is a museum dedicated to his life, times and work.[citation needed]",What was Avicenna's birthplace?,"{'text': ['Bukhara'], 'answer_start': [59]}"
2,572f69a8947a6a140053c927,Han_dynasty,"Apart from the passing of noble titles or ranks, inheritance practices did not involve primogeniture; each son received an equal share of the family property. Unlike the practice in later dynasties, the father usually sent his adult married sons away with their portions of the family fortune. Daughters received a portion of the family fortune through their marriage dowries, though this was usually much less than the shares of sons. A different distribution of the remainder could be specified in a will, but it is unclear how common this was.",What type of document could be produced to distribute some of an inheritance?,"{'text': ['a will'], 'answer_start': [500]}"
3,572cabc8f182dd1900d7c81e,Law_of_the_United_States,"After the President signs a bill into law (or Congress enacts it over his veto), it is delivered to the Office of the Federal Register (OFR) of the National Archives and Records Administration (NARA) where it is assigned a law number, and prepared for publication as a slip law. Public laws, but not private laws, are also given legal statutory citation by the OFR. At the end of each session of Congress, the slip laws are compiled into bound volumes called the United States Statutes at Large, and they are known as session laws. The Statutes at Large present a chronological arrangement of the laws in the exact order that they have been enacted.",What is the United States Statutes at Large?,"{'text': ['a chronological arrangement of the laws in the exact order that they have been enacted'], 'answer_start': [562]}"
4,572798f7708984140094e1dd,Switzerland,"Traditionally, Switzerland avoids alliances that might entail military, political, or direct economic action and has been neutral since the end of its expansion in 1515. Its policy of neutrality was internationally recognised at the Congress of Vienna in 1815. Only in 2002 did Switzerland become a full member of the United Nations and it was the first state to join it by referendum. Switzerland maintains diplomatic relations with almost all countries and historically has served as an intermediary between other states. Switzerland is not a member of the European Union; the S

## 预处理数据

In [16]:
# AutoTokenizer 是transformers库中的一个重要组件，用于加载和使用不同的预训练模型的标记化器（Tokenizer）。
from transformers import AutoTokenizer

In [17]:
# 想要使用的模型可以从 from_pretrained() 方法的预训练模型的名称或路径中推测出来。
# 加载预训练的模型的分词器。
# AutoTokenizer.from_pretrained 通过输入的分词器名称或路径，查找到对应的分词器。
# 该方法根据指定的模型检查点（model_checkpoint）自动加载与之相对应的预训练分词器。这个检查点通常是一个模型的名称。
# 如bert-base-uncased、gpt-2等。
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

##### 以下断言确保我们的 Tokenizers 使用的是 FastTokenizer 分词器（Rust 实现，速度和功能性上有一定优势）。

In [19]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

可以在大模型表上查看哪种类型的模型具有可用的快速标记器，哪种类型没有。

可以直接在两个句子上调用此分词器（一个用于答案，一个用于上下文）：


双句子输入：当你传递两个句子给分词器时，它通常会将这两个句子视为一对句子。这在一些任务中很常见，比如问答任务或句子关系判断任务。

分词处理：分词器会将每个句子分割成更小的单元（词、子词或符号）。对于某些模型（如BERT），它还会添加特殊的标记，如 [CLS] 和 [SEP]，以分隔句子并标记句子的开始和结束。

输出：分词器的输出通常包含几个组件，最主要的是 input_ids（分词后的词汇表中的ID序列），以及可能的是 attention_mask（标识哪些ID是有意义的，哪些是填充的）和 token_type_ids（标识每个令牌属于哪个句子）。


In [20]:
# 获得编码
token = tokenizer("what is your name?", "my name is AnMin")

In [22]:
token

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 2019, 10020, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
token.keys()

dict_keys(['input_ids', 'attention_mask'])

In [23]:
# decode 解码，根据tokenID映射回原始的句子
tokenizer.decode(token['input_ids'])

'[CLS] what is your name? [SEP] my name is anmin [SEP]'

In [24]:
# convert_ids_to_tokens 根据指定得input_ids映射回原始的词
tokenizer.convert_ids_to_tokens(token['input_ids'][:6])

['[CLS]', 'what', 'is', 'your', 'name', '?']

### Tokenizer 进阶操作

**在问答预处理中的一个特定问题是如何处理非常长的文档。**

在其他任务中，当文档的长度超过模型最大句子长度时，我们通常会截断它们，但在这里，删除上下文的一部分可能会导致我们丢失正在寻找的答案。

为了解决这个问题，我们允许数据集中的一个（长）示例生成多个输入特征，每个特征的长度都小于模型的最大长度（或我们设置的超参数）。

In [28]:
# The maximum length of a feature (question and context)
# 一个特征的最大长度(问题和上下文)
max_length = 384

# The authorized overlap between two part of the context when splitting it is needed.
# 需要拆分上下文时，上下文的两个部分之间的授权重叠。
doc_stride = 128

#### 超出最大长度的文本数据处理

从训练集中找出一个超过最大长度（384）的文本 作为演示文本：

In [29]:
# 找到一个长度大于384的文本作为例子,问题和文本加起来token小于384个
for i, example in enumerate(dataset["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
# 挑选出来超过384（最大长度）的数据样例
example = dataset["train"][i]

In [26]:
example

{'id': '5733caf74776f4190066124c',
 'title': 'University_of_Notre_Dame',
 'context': "The men's basketball team has over 1,600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 NCAA tournaments. Former player Austin Carr holds the record for most points scored in a single game of the tournament with 61. Although the team has never won the NCAA Tournament, they were named by the Helms Athletic Foundation as national champions twice. The team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending UCLA's record 88-game winning streak in 1974. The team has beaten an additional eight number-one teams, and those nine wins rank second, to UCLA's 10, all-time in wins against the top team. The team plays in newly renovated Purcell Pavilion (within the Edmund P. Joyce Center), which reopened for the beginning of the 2009–2010 season. The team is coached by Mike Brey, who, as of the 2014–15 season, his fifteenth at Notre

In [30]:
# 获得问题和文本的token长度
len(tokenizer(example["question"], example["context"])["input_ids"])

396

#### 文本预处理-截断上下文后 不保留超出部分 丢弃截断后的数据

truncation 参数的选项 用于设置截取的方式

- True 或 'longest_first': 这是默认选项。当输入长度超过最大长度限制时，会从最长的输入序列开始截断，直到总长度符合要求。如果有多个序列（例如，在文本对任务中），则首先截断最长的序列，如果需要，再截断第二长的序列，依此类推。

- 'only_first': 当处理一对序列时（例如，在问答任务或文本对比任务中），这个选项仅截断第一个序列（通常是问题或假设），而保留第二个序列（通常是上下文或前提）的完整性。

- 'only_second': 与'only_first'相反，这个选项仅截断第二个序列，保留第一个序列的完整性。在某些问答任务中，这可能有助于确保问题的完整性。

- False: 不进行任何截断。如果输入序列超过了模型的最大长度限制，将会抛出错误。这个选项适用于确保输入数据完全符合模型要求的场景。

In [35]:
# truncation截断 方式一 从最长的输入序列开始截断，直到总长度符合要求
token = tokenizer(example["question"],example["context"],
              max_length = max_length,
              truncation = True)
print(len(token['input_ids']))
tokenizer.decode(token['input_ids'])

384


"[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at not

In [37]:
# truncation截断 方式二 
# only_first 截断第一个序列（通常是问题或假设），保留第二个序列（通常是上下文或前提）的完整性。
token = tokenizer(example["question"],example["context"],
              max_length = max_length,
              truncation = "only_first")
print(len(token['input_ids']))
tokenizer.decode(token['input_ids'])

384


"[CLS] how many [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at notre dame, has achieved a 332 - 165 record. in 2009 the

In [38]:
# truncation截断
# 'only_second': 仅截断第二个序列，保留第一个序列的完整性。在某些问答任务中，这可能有助于确保问题的完整性。
token = tokenizer(example["question"],example["context"],
              max_length = max_length,
              truncation = "only_second")
print(len(token['input_ids']))
tokenizer.decode(token['input_ids'])


384


"[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at not

#### 关于截断后产生的超出部分的文本的策略（不丢弃）

- 直接截断超出部分: 当 truncation=`only_second` 时，截断第二个序列，保留第一个序列的完整性
- 仅截断上下文（context），保留问题（question）：设置 `return_overflowing_tokens=True` 和设置`stride`长度时 stride为截断后要补偿的
- 当你设置 return_overflowing_tokens=True 时，分词器会返回一个额外的字段，通常称为 overflowing_tokens 或类似名称。

In [39]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

使用此策略截断后，Tokenizer 将返回多个 `input_ids` 列表。

In [53]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 29]

In [43]:
tokenized_example['overflow_to_sample_mapping']

[0, 0]

In [60]:
# print(dataset['train']['question']+dataset['train']['context'])
# 可以看到截断后的超出最大长度的部分像前补全到stride设置大小 最终长度为stride+超出部分的长度大小
# 解码两个输入特征，可以看到重叠的部分：
print(tokenizer.decode(tokenized_example["input_ids"][0]))
print(tokenizer.decode(tokenized_example["input_ids"][1]))

[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at notr

In [44]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    # stride=doc_stride
)

In [46]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

可以看到没有设置stride的时候，则默认与max_length长度相同 每次阶段出来的都是满足384个

In [52]:
print(tokenizer.decode(tokenized_example['input_ids'][0])[:100])
print(tokenizer.decode(tokenized_example['input_ids'][1])[:100])

[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team 
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the most by the fighting i


#### 使用 offsets_mapping 获取原始的 input_ids

设置 `return_offsets_mapping=True`，将使得截断分割生成的多个 input_ids 列表中的 token，通过映射保留原始文本的 input_ids。

当 return_offsets_mapping=True 时，分词器会为每个令牌返回一个元组，表示该令牌在原始未分词文本中的字符级偏移量。这个元组的形式通常是 (start, end)，

其中 start 是令牌在原文中的开始位置，end 是结束位置（不包括该位置）。这里的偏移指的是 字母级别的偏移

如下所示：第一个标记（[CLS]）的起始和结束字符都是（0, 0），因为它不对应问题/答案的任何部分，然后第二个标记与问题(question)的字符0到3相同.



In [82]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)

In [68]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [83]:

start, end = tokenized_example['offset_mapping'][0][1]
example["question"][start:end]

'How'

In [84]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


In [85]:
second_token_id = tokenized_example["input_ids"][0][2]
offsets = tokenized_example["offset_mapping"][0][2]
print(tokenizer.convert_ids_to_tokens([second_token_id])[0], example["question"][offsets[0]:offsets[1]])

many many


#### convert_ids_to_tokens 和 decoder 区别：
#### convert_ids_to_tokens：可以是token序列号
#### decoder：是在整个字符串级别上进行的 不能多个


In [58]:
# 问题
example["question"]

"How many wins does the Notre Dame men's basketball team have?"

借助`sequence_ids`方法，我们可以方便的区分token的来源编号：

- 对于特殊标记：返回None，
- 对于正文Token：返回句子编号（从0开始编号）。

综上，现在我们可以很方便的在一个输入特征中找到答案的起始和结束 Token。

In [55]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [56]:
len(sequence_ids)

384

In [63]:
# 获取回答结果
answers = example['answers']
print(answers)


{'text': ['over 1,600'], 'answer_start': [30]}


In [64]:
# 获取回答的结果在文本上的起始地址（以字符为单位的）
start_char = answers['answer_start'][0]
# 获取回答的结果在文本上的结束地址（以字符为单位的
end_char = start_char + len(answers['text'][0])

计算文本内容在tokenized_example中的位置

In [60]:
# 当前span在文本中的起始标记索引。sequence_id为token的来源编号
# 判断是第一句的token的个数
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# 当前span在文本中的结束标记索引。计算是第一句的长度
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
token_start_index, token_end_index

(16, 382)

In [ ]:
# 检测答案是否超出span范围（如果超出范围，该特征将以CLS标记索引标记）。
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # 将token_start_index和token_end_index移动到答案的两端。
    # 注意：如果答案是最后一个单词，我们可以移到最后一个标记之后（边界情况）。
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("答案不在此特征中。")

In [ ]:
# 通过查找 offset mapping 位置，解码 context 中的答案 
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
# 直接打印 数据集中的标准答案（answer["text"])
print(answers["text"][0])

#### 关于填充的策略

- 对于没有超过最大长度的文本，填充补齐长度。
- 对于需要左侧填充的模型，交换 question 和 context 顺序

In [67]:
pad_on_right = tokenizer.padding_side == "right"

### 整合以上所有预处理步骤

让我们将所有内容整合到一个函数中，并将其应用到训练集。

针对不可回答的情况（上下文过长，答案在另一个特征中），我们为开始和结束位置都设置了cls索引。

如果allow_impossible_answers标志为False，我们还可以简单地从训练集中丢弃这些示例。

In [ ]:
def prepare_train_features(examples):
    ''' example datasetlaoder 后的数据'''
    # 一些问题的左侧可能有很多空白字符，这对我们没有用，而且会导致上下文的截断失败
    # （标记化的问题将占用大量空间）。因此，我们删除左侧的空白字符。
    examples["question"] = [q.lstrip() for q in examples["question"]]
    # 获取token编码，这里注意需要判断是填充还是截取
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # 由于一个示例可能给我们提供多个特征（如果它具有很长的上下文），我们需要一个从特征到其对应示例的映射。这个键就提供了这个映射关系。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # 偏移映射将为我们提供从令牌到原始上下文中的字符位置的映射。这将帮助我们计算开始位置和结束位置。
    offset_mapping = tokenized_examples.pop("offset_mapping")
    # 让我们为这些示例进行标记！
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        
